In [1]:
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [15]:
df = pd.read_excel('../data/RFM 分析.xlsx')
df.head()

,COMPANY,CUSTOMERNAME,CUSTOMERTYPE,DATE,AMOUNT,MONEY
0,广州,广州璟田食品有限公司,E类,2010-02-17,3953,8036611
1,宁波,武汉鑫一航食品有限公司,E类,2012-02-17,2996,7750870
2,南京,南京运联食品有限公司,E类,2012-02-17,3633,7501937
3,南京,南京运联食品有限公司,E类,2012-02-17,2854,6109309
4,宁波,武汉鑫一航食品有限公司,E类,2001-02-18,2992,6043166


In [36]:
# 计算每条数据离现在最近一次消费时间（R）
import datetime
def R(s):
    s = datetime.datetime.strptime(s,'%Y-%m-%d')
    now = str(datetime.datetime.now()).split()[0]
    now = datetime.datetime.strptime(now,'%Y-%m-%d')
    r = now - s
    r = int(str(r).split()[0])
    return r
df['R'] = df['DATE'].map(R)
# 分组计算每个【COMPANY】，最小【R】，计数【AMOUNT】，合计【MONEY】
rfm = df.groupby(['COMPANY','CUSTOMERNAME','CUSTOMERTYPE']).agg({'R':'min','AMOUNT':'count','MONEY':'sum'}).reset_index()
rfm.rename(columns={'AMOUNT':'F','MONEY':'M'},inplace=True)
# 计算平均 RFM
rfm['avg_R'] = round(rfm['R'].mean(),0)
rfm['avg_F'] = round(rfm['F'].mean(),0)
rfm['avg_M'] = round(rfm['M'].mean(),0)
# 大于平均值标记 1
rfm['flag_R'] = rfm['R'] < rfm['avg_R']
rfm['flag_R'] = rfm['flag_R'].map(lambda x: '1' if x else '0')
rfm['flag_F'] = rfm['F'] > rfm['avg_F']
rfm['flag_F'] = rfm['flag_F'].map(lambda x: '1' if x else '0')
rfm['flag_M'] = rfm['M'] > rfm['avg_M']
rfm['flag_M'] = rfm['flag_M'].map(lambda x: '1' if x else '0')
# 合并 RFM 指标，分组计算每种类型平均值
rfm['RFM'] = rfm['flag_R'] + rfm['flag_F'] + rfm['flag_M']
rfm = round(rfm.groupby('RFM').mean()[['R','F','M']].reset_index(),0)
rfm

,RFM,R,F,M
0,000,5533.0,3.0,273007.0
1,001,5807.0,7.0,3549416.0
2,010,5413.0,30.0,436867.0
3,011,5445.0,45.0,12299335.0
4,100,3470.0,6.0,205551.0
5,101,3317.0,12.0,3831896.0
6,110,3152.0,64.0,460734.0
7,111,3177.0,69.0,10912394.0


In [54]:
from pyecharts import options as opts
from pyecharts.charts import Radar


v1 = [[5533.0,3.0,273007.0]]
v2 = [[5807.0,7.0,3549416.0]]
v3 = [[5413.0,30.0,436867.0]]
v4 = [[5445.0,45.0,12299335.0]]
v5 = [[3470.0,6.0,205551.0]]
v6 = [[3317.0,12.0,3831896.0]]
v7 = [[3152.0,64.0,460734.0]]
v8 = [[3177.0,69.0,10912394.0]]
c = (
    Radar()
    .add_schema(
        schema=[
            opts.RadarIndicatorItem(name="R", max_=6000),
            opts.RadarIndicatorItem(name="F", max_=75),
            opts.RadarIndicatorItem(name="M", max_=12299335.0),
        ]
    )
    .add("一般挽留客户", v1,color="#ff4d4f")
    .add("重要挽留客户", v2,color="#ffc069")
    .add("一般保持客户", v3,color="#ffec3d")
    .add("重要保持客户", v4,color="#a0d911")
    .add("一般发展客户", v5,color="#13c2c2")
    .add("重要发展客户", v6,color="#2f54eb")
    .add("一般价值客户", v7,color="#531dab")
    .add("重要价值客户", v8,color="#c41d7f")
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        legend_opts=opts.LegendOpts(pos_left='0%')
    )
)
c.render('../output/RFM 雷达图.html')
c.render_notebook()

D:\programming\dev_env\anaconda\Anaconda3\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)
